## Notebook pour la récupération des commentaires des utilisateurs sur IMBD via Selenium

In [9]:
#pip install selenium

In [1]:
import time
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
data_bo = pd.read_csv("data_bo.csv")

In [ ]:
data_bo['Titre']

In [77]:
data_bo.iloc[0,0] = 'Avatar (2009)'
data_bo.iloc[8,0] = "Le Roi lion (1994)"
data_bo.iloc[9,0] = "Avengers (2012)"

0                                         Avatar (2009)
1                                     Avengers: Endgame
2                             Avatar : La Voie de l'eau
3                                               Titanic
4        Star Wars, épisode VII : Le Réveil de la Force
5                                Avengers: Infinity War
6                               Spider-Man: No Way Home
7                                        Jurassic World
8                                           Le Roi lion
9                                              Avengers
10                                   Fast and Furious 7
11                                   Top Gun : Maverick
12                                La Reine des neiges 2
13                                               Barbie
14                            Avengers : L'Ère d'Ultron
15                            Super Mario Bros. le film
16                                        Black Panther
17    Harry Potter et les Reliques de la Mort : 

!!! attention !!! Il faut télécharger un "chromedriver" de la même version que votre version chrome, et le mettre à coté de ce fichier jupyter dans les fichiers.

In [60]:
def scrape_imdb_reviews(url, num_iterations=3, wait_time=20):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, wait_time)
    
    # Locate the span element with the specified class
    span_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bde20123-1")))

    # Get the text content of the span element
    note = span_element.text

    #  Print or use the extracted text
    print(note)
    
    time.sleep(4)

    # Get the current scroll position
    current_scroll_position = driver.execute_script("return window.scrollY;")

    # Scroll down by 200 pixels from the current position
    scroll_by_pixels = 400
    new_scroll_position = current_scroll_position + scroll_by_pixels
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    
    time.sleep(2)
    
    user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

    # Cliquez sur le bouton
    user_comments_button.click()
    wait = WebDriverWait(driver, wait_time)
    
    
    #cherche le bouton "Hide Spoilers"
    spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
    # Cliquez sur le bouton
    spoiler_button.click()
    
    # Clique sur le bouton "Load More" plusieurs fois
    for i in range(num_iterations):
        load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
        load_more_button.click()
        driver.implicitly_wait(3)

    time.sleep(3)

    # Trouve les commentaires par classe CSS
    reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

    # Imprime les commentaires pour débugger (les trois premiers commentaires)
    
    reviews_txt = []
    for review in reviews:
        reviews_txt.append(review.text) 

    # Imprime le nombre total de commentaires récupérés
    print(f"Nombre total de commentaires : {len(reviews)}")

    time.sleep(2)

    # Ferme le navigateur
    driver.quit()
    
    return reviews_txt, note

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

Le lien "hidespoil"ci-dessous est ce lien:https://www.imdb.com/title/tt0499549/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0
où l'on a coché la case "hide spoiler" ce qui retire automatiquement de la liste les commentaires avec la banière warning spoil. Ce qui nous permet de ne pas avoir d'élément vide dans notre liste. 
J'ai donc modifié la fonction pour que le robot clique automatiquement sur le bouton "hide spoilers" ce qui permet d'aller vers le 2ème lien à partir du 1er lien automatiquement.

In [59]:
url_avatar = "https://www.imdb.com/title/tt0499549/"
reviews_avatar = scrape_imdb_reviews(url_avatar, num_iterations=3, wait_time=20)

7,9
["I was waiting for this day for the longest time. I was a kid back in 2009 when this movie released. So never got to watch it back then. But now when it rereleased I booked the first day show to a IMAX show and oh boy was I blown away! This is nothing short of a masterpiece! It's beyond belief how a film like this could've been made. Every scene, every shot is perfection. You are transferred to a different world and become so engrossed in the film. Never have I attended a movie where at the end of it people got up from their seats and started clapping! Last time this happened was after Infinity War. That movie too was a damn brilliant one. I'm from India and this is legit rare, where people go crazy, so crazy after any English film. This was one experience that I will never forget. I'm definitely going for it again next weekend cuz just once was not enough.\n\nEvery human on the planet needs to experience this magnificent work of art!", "Forgive me, I'm going to jump from professi

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

In [63]:
#vérification que nous avons pas de commentaire "vide"
X = np.array([len(review) for review in reviews_avatar_2])
len(X)
X

array([ 945, 5506, 2389, 4167, 1033, 2754, 5675, 1922, 2022, 1352, 3585,
       3852, 2429,  850,  997, 1305,  715, 1047,  184, 1724,  824, 1275,
       2433, 4197, 1014, 5140, 1856, 2332, 2867, 2324, 5172, 1030, 4866,
       2689,  184, 2714, 2715, 4956, 1662, 2300,  820, 4488,  767, 2884,
       2559, 4741, 2492, 1159, 1759, 1268, 5574, 1679, 3027,  765,  992,
       4081,  426, 1418, 5216, 1440,  820,  753,  397,  296,  550, 1259,
        802,  743, 1211, 2292,  792, 1416, 1578, 2094, 3903,  403,  972,
       1263, 5415, 1049, 3298,  903,  484, 2565,  781,  477, 1482, 1119,
        719, 2289, 3680,  817, 2203, 1361,  769,  254, 1792, 1039, 3628,
        227])

In [61]:
#Test pour un autre film
url_cars = "https://www.imdb.com/title/tt0317219/?ref_=nv_sr_srsg_0_tt_8_nm_0_q_cars"
reviews_cars = scrape_imdb_reviews(url_cars, num_iterations=3, wait_time=20)

7,2
Nombre total de commentaires : 100


In [75]:
def scrap_review_all(movies, wait_time=10, num_iterations=3):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    dico_all_comments = {}
    driver.get("https://www.google.com")
    wait = WebDriverWait(driver, wait_time)
    # Attendre que le bouton soit présent
    button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))
        
    # Cliquez sur le bouton
    button.click()
    
    for movie in movies:
        # Ouvrir Google
        driver.get("https://www.google.com")
        wait = WebDriverWait(driver, wait_time)
        
        
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys("imdb title " + movie )
        search_box.send_keys(Keys.RETURN)
        
        # Attente que les résultats de recherche soient chargés
        wait.until(EC.presence_of_element_located((By.ID, "search")))
        
        time.sleep(2)
        
        # Trouver le premier lien dans les résultats de recherche et cliquer dessus
        first_link = driver.find_element(By.CSS_SELECTOR, "h3")
        first_link.click()
        
        time.sleep(2)
        
        # Locate the span element with the specified class
        span_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bde20123-1")))

        # Get the text content of the span element
        note = span_element.text

        #  Print or use the extracted text
        print(note)
    
        time.sleep(4)

        # Get the current scroll position
        current_scroll_position = driver.execute_script("return window.scrollY;")

           # Scroll down by 200 pixels from the current position
        scroll_by_pixels = 400
        new_scroll_position = current_scroll_position + scroll_by_pixels
        driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    
        time.sleep(2)
    
        user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

         # Cliquez sur le bouton
        user_comments_button.click()
        wait = WebDriverWait(driver, wait_time)
        
        time.sleep(1)
    
    
         #cherche le bouton "Hide Spoilers"
        spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
        # Cliquez sur le bouton
        spoiler_button.click()
        
 
    
        # Clique sur le bouton "Load More" plusieurs fois
        for i in range(num_iterations):
            load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
            load_more_button.click()
            time.sleep(2)

        time.sleep(4)

        # Trouve les commentaires par classe CSS
        reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

        
        reviews_txt = [review.text for review in reviews]
        dico_all_comments[movie] = [reviews_txt, note]
        
        time.sleep(4)
    
    return dico_all_comments

In [76]:
comments_all = scrap_review_all(list(data_bo.iloc[:,0]), wait_time=20)

7,9
8,4
7,6
7,9
7,8
8,4
8,2
6,9
6,8
8,0
7,1
8,3
6,8
7,0
7,3
7,1
7,3
8,1
6,9
7,4
6,1
6,4


TimeoutException: Message: 
